# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [97]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [98]:
cities_df = pd.read_csv('../WeatherPy/cities.csv', index_col= 0)
cities_df

,City,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon,Country,Date
0,nampula,78.75,53,40,9.22,-15.1165,39.2666,MZ,1651674304
1,albany,51.64,91,100,1.99,42.6001,-73.9662,US,1651674194
2,chokurdakh,11.57,94,100,3.09,70.6333,147.9167,RU,1651674305
3,port augusta,54.00,63,19,12.71,-32.5000,137.7667,AU,1651674305
4,komsomolskiy,72.32,60,75,9.22,40.4272,71.7189,UZ,1651673834
...,...,...,...,...,...,...,...,...,...
585,belyy yar,42.85,40,0,4.47,53.6039,91.3903,RU,1651674526
586,port hawkesbury,57.90,66,0,6.91,45.6169,-61.3485,CA,1651674527
587,arroio do meio,67.87,100,100,5.79,-29.4011,-51.9450,BR,1651674527
588,lompoc,48.94,89,0,6.91,34.6391,-120.4579,US,1651674528


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [99]:
locations = cities_df[['Lat', 'Lon']]

gmaps.configure(api_key=g_key)

figure_layout = {'width': '700px',
                'height': '500px',
                'border': '1 px solid black',
                'padding': '1px'}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights= cities_df['Humidity (%)'], dissipating = False, 
                                 point_radius= 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1 px solid black', height='500px', padding='1px', width='700px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [100]:
#Criteria Max Temp between 75 and 95, humidity less than 90% and cloudiness less than 30%
ideal_cities_df = cities_df.loc[(cities_df['Max Temp (F)'] >= 75) & (cities_df['Max Temp (F)'] <= 95)
                               & (cities_df['Humidity (%)'] <= 90) & (cities_df['Cloudiness (%)'] <= 30)]
ideal_cities_df

,City,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon,Country,Date
7,isangel,76.73,81,0,3.33,-19.5500,169.2667,VU,1651673833
26,arraial do cabo,87.66,55,20,12.66,-22.9661,-42.0278,BR,1651673747
38,lata,78.08,36,2,9.48,40.1629,-8.3327,PT,1651674316
50,akyab,84.31,63,3,3.76,20.1500,92.9000,MM,1651674321
61,namibe,85.08,52,0,11.54,-15.1961,12.1522,AO,1651674324
...,...,...,...,...,...,...,...,...,...
544,yinchuan,79.34,7,4,4.12,38.4681,106.2731,CN,1651674512
549,nathdwara,92.39,15,0,11.50,24.9333,73.8167,IN,1651674513
561,luxor,93.25,18,5,10.36,25.6989,32.6421,EG,1651674518
564,las matas de farfan,82.02,54,4,5.77,18.8731,-71.5166,DO,1651674519


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [101]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {"type": "hotel",
         "key":g_key,
         "radius": "5000",
         "rankby": "prominence"}

hotel_df = ideal_cities_df
hotel_df['Hotel Name'] = ""
hotel_df.reset_index(drop= True, inplace= True)
hotel_df

C:\Users\wawil\AppData\Local\Temp/ipykernel_36632/2475945576.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon,Country,Date,Hotel Name
0,isangel,76.73,81,0,3.33,-19.5500,169.2667,VU,1651673833,
1,arraial do cabo,87.66,55,20,12.66,-22.9661,-42.0278,BR,1651673747,
2,lata,78.08,36,2,9.48,40.1629,-8.3327,PT,1651674316,
3,akyab,84.31,63,3,3.76,20.1500,92.9000,MM,1651674321,
4,namibe,85.08,52,0,11.54,-15.1961,12.1522,AO,1651674324,
...,...,...,...,...,...,...,...,...,...,...
62,yinchuan,79.34,7,4,4.12,38.4681,106.2731,CN,1651674512,
63,nathdwara,92.39,15,0,11.50,24.9333,73.8167,IN,1651674513,
64,luxor,93.25,18,5,10.36,25.6989,32.6421,EG,1651674518,
65,las matas de farfan,82.02,54,4,5.77,18.8731,-71.5166,DO,1651674519,


In [102]:
params['location'] = str(hotel_df['Lat'][0]) + "," + str(hotel_df['Lon'][0])
response = requests.get(base_url, params).json()
result = response['results']
print(result[1]['name'])

Tanna Lodge


In [103]:
for index, row in hotel_df.iterrows():
    
    params['location'] = str(row['Lat']) + "," + str(row['Lon'])
    
    print(f"Retrieving Results for Index {index}: {row['City']}")
    response = requests.get(base_url, params=params).json()
    result = response['results']
    
    try:

        print(f"Closest hotel is {result[1]['name']}")
        hotel_df.loc[index, 'Hotel Name'] = result[1]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result...skipping")
        hotel_df.drop(index, inplace= True)

Retrieving Results for Index 0: isangel
Closest hotel is Tanna Lodge
Retrieving Results for Index 1: arraial do cabo


C:\Users\wawil\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is Pousada Porto Praia
Retrieving Results for Index 2: lata
Closest hotel is Casa Maquia
Retrieving Results for Index 3: akyab
Closest hotel is Global
Retrieving Results for Index 4: namibe
Closest hotel is Pavilhão Welwitschia Mirabilis
Retrieving Results for Index 5: manzanillo
Closest hotel is Religiosas Hijas de María Auxiliadora,Hermanas Salesianas
Retrieving Results for Index 6: karratha
Closest hotel is ibis Styles Karratha
Retrieving Results for Index 7: vila velha
Closest hotel is Hotel Golden Tulip Porto Vitória
Retrieving Results for Index 8: black river
Closest hotel is Idlers' Rest Beach Hotel
Retrieving Results for Index 9: alofi
Closest hotel is Niue Post
Retrieving Results for Index 10: carutapera
Closest hotel is Cartório 1º Ofício Comarca de Carutapera
Retrieving Results for Index 11: vao
Closest hotel is Hotel Kou-Bugny
Retrieving Results for Index 12: calvinia
Closest hotel is Die Lemoenhuis
Retrieving Results for Index 13: san patricio
Closest hotel i

In [104]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [105]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='1 px solid black', height='500px', padding='1px', width='700px'))